In [1]:
card_names = [
    'Snow Covered Island', 
    'Fabled Passage', 
    'Once Upon a Time', 
    'Murderous Rider // Swift End', 
    'Questing Beast', 
    'Oko, Thief of Crowns'
]
file_names = [card_name.replace('/', '').replace(',', '').replace('  ', '_').replace(' ', '_') for card_name in card_names]
index = 5
file_name = file_names[index]
card_name = card_names[index]
file_names

['Snow_Covered_Island',
 'Fabled_Passage',
 'Once_Upon_a_Time',
 'Murderous_Rider_Swift_End',
 'Questing_Beast',
 'Oko_Thief_of_Crowns']

In [2]:
'''
loading the pickled data to plot, produce it in a format ready to be consumed by fusion
'''
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 
import matplotlib 
from sqlalchemy import create_engine
import pickle
from scipy.stats import zscore

%matplotlib inline 
idx = pd.IndexSlice

In [3]:
with open('./pickles/df.pickle', 'rb') as handle:
    df = pickle.load(handle)

#javascript code
var flask_data = JSON.parse('{{ data_price_distribution_lastest_time_stamp | tojson | safe}}');
var dataSource = {
    chart: {
        caption: "Item Amount Distribution",
        subcaption: "From latest timestamp",
        xaxisname: "Price",
        yaxisname: "Amount",
        numbersuffix: "units",
        theme: "fusion"
    },
    data: flask_data
};
FusionCharts.ready(function() {
    var chart = new FusionCharts({
        type: 'column2d',
        renderAt: 'chart-container-price_distribution_lastest_time_stamp',
        width: '100%',//100% 700
        height: '700%',//300% 400
        dataFormat: 'json',
        dataSource
    });
    chart.render();

# web_plot_price_distribution_lastest_time_stamp_all

In [4]:
for card_name, file_name in zip(card_names, file_names):


    df_ = df.loc[idx[str(df.index[-1][0]), card_name], ['item_price', 'item_amount']].\
        reset_index(drop=True)
    df_repeated = pd.DataFrame(np.repeat(df_.item_price.values, df_.item_amount.values), columns=['item_price'])
    df_amount = df_.groupby('item_price').sum().reset_index()
    
    df_filtered = df_repeated
    for i in range(5):
        df_filtered = df_filtered[(zscore(df_filtered.item_price) < 3)]
        print('shape', df_filtered.shape)
        print('mean', df_filtered.item_price.mean())
        print('median', df_filtered.item_price.median())
        print('std', df_filtered.item_price.std())
        display(df_filtered)
        print('---')


    df_ = df_amount.loc[df_amount.item_price.isin(df_filtered.item_price)]
    
    plot_constant = 25
    start = 0.0
    stop = df_.item_price.max()
    step = round(df_.item_price.max()/plot_constant, 2)
    item_price_ = np.arange(start, stop, step)
    df__ = pd.DataFrame([(item_price_[i], 0) for i in range(len(item_price_))], columns=['item_price', 'item_amount'])
    df_dummy = df__.append(df_).sort_values(by=['item_price']).reset_index(drop=True)

    '''
    let's try to plot with the interpolated points
    Now it runs!! and we have the data that is relevant to us
    '''
    df_plot = df_dummy.loc[df_dummy.item_price <= df_filtered.item_price.max()]

    '''
    Finally, lets bin the data so that we can actually see the bars

    TODO: come up with a way to set the bin span automatically
    '''

    df_plot_ = df_plot.set_index('item_price')

    # the range of each bin
    bin_span = round(df_plot_.index.max()/plot_constant, 2)

    df_plot_binned = df_plot_.\
        groupby(pd.cut(df_plot_.index, np.arange(0, df_plot_.index.max() + bin_span, bin_span))).sum()

    df_pickle = df_plot_binned.reset_index().copy()
    df_pickle = df_pickle.rename(columns={'index':'label', 'item_amount':'value'})
    df_pickle = df_pickle.astype({'label':str, 'value':str})
    df_pickle.loc[:, 'label'] = df_pickle.loc[:, 'label'].str[1:-1].to_frame()
    list_pickle = list(df_pickle.T.to_dict().values())

    with open('./app/app/pickles/' + file_name + '_price_distribution_lastest_time_stamp.pickle', 'wb') as handle:
        pickle.dump(list_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

shape (10673, 1)
mean 1.5678150473156565
median 0.4825
std 3.3286225154901734


,item_price
0,0.02
1,0.05
2,0.05
3,0.08
4,0.09
...,...
10668,29.99
10669,29.99
10670,29.99
10671,30.00


---
shape (10461, 1)
mean 1.2319902494981358
median 0.47
std 2.1982982627931484


,item_price
0,0.02
1,0.05
2,0.05
3,0.08
4,0.09
...,...
10456,11.00
10457,11.00
10458,11.00
10459,11.00


---
shape (9955, 1)
mean 0.8293962832747362
median 0.45
std 1.2988663720287894


...

---
shape (979, 1)
mean 51.80889683350358
median 46.7
std 14.00030269661009


,item_price
0,34.5
1,35.0
2,35.0
3,35.0
4,35.5
...,...
974,100.0
975,100.0
976,100.0
977,100.0


---
shape (953, 1)
mean 50.51379853095488
median 45.95
std 11.750720699081098


,item_price
0,34.50
1,35.00
2,35.00
3,35.00
4,35.50
...,...
948,88.69
949,89.00
950,89.00
951,89.99


---
